In [1]:
import warnings
warnings.filterwarnings('ignore')
import sqlite3
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import undetected_chromedriver as uc

In [3]:
# Susirenkame pigu.lt šaldytuvų nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 19):
    url = f'https://pigu.lt/lt/saldytuvai?page={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    pigiukai = bs.find_all('div', {'class': 'c-product-card__title'})

    for prekes in pigiukai:
        try:
            nuorodos.append(prekes.find('a')['href'])
        except:
            pass
        
driver.close()

In [6]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('pigusaldytuvai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists saldytuvai
(
nuoroda text,
kaina text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(0, len(nuorodos)):
    try:
        url = f'{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')

        stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'pigusaldytuvai.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]

        try:
            kaina = bs.find('div', {'class': 'c-price h-price--xx-large h-price'})
            kaina = kaina.text.replace(' €','')[:-3].strip()
            duomenys['kaina'] = kaina
        except:
            pass

        try:
            kaina = bs.find('div', {'class': 'c-price h-price--xx-large h-price--new'})
            kaina = kaina.text.replace(' €','')[:-3].strip()
            duomenys['kaina'] = kaina
        except:
            pass
    
        renku = bs.find('div', {'class': 'h-row h-pl--8 h-pr--8'})
        radau= renku.find_all('tr')

        for i in radau:
            aha = i.find_all('td')
            duomenys[aha[0].text.strip().lower()]= aha[1].text.strip().lower()
                  
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'pigusaldytuvai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'saldytuvai', 'pigusaldytuvai.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'pigusaldytuvai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'saldytuvai', 'pigusaldytuvai.db')
        writeMany(duomenys, 'saldytuvai', 'pigusaldytuvai.db')
    

driver.close()

In [2]:
# Susirenkame pigu.lt dronų nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 6):
    url = f'https://pigu.lt/lt/kompiuterine-technika/dronai?f[51959][5261711]=dronas&page={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    pigiukai = bs.find_all('div', {'class': 'c-product-card__title'})

    for prekes in pigiukai:
        try:
            nuorodos.append(prekes.find('a')['href'])
        except:
            pass
        
driver.close()

In [3]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('pigudronai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists dronai
(
nuoroda text,
kaina text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(0, len(nuorodos)):
    try:
        url = f'{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')

        stulpeliai, *_ =zip(*getColNamesList('dronai', 'pigudronai.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]

        try:
            kaina = bs.find('div', {'class': 'c-price h-price--xx-large h-price'})
            kaina = kaina.text.replace(' €','')[:-3].strip()
            duomenys['kaina'] = kaina
        except:
            pass

        try:
            kaina = bs.find('div', {'class': 'c-price h-price--xx-large h-price--new'})
            kaina = kaina.text.replace(' €','')[:-3].strip()
            duomenys['kaina'] = kaina
        except:
            pass
    
        renku = bs.find('div', {'class': 'h-row h-pl--8 h-pr--8'})
        radau= renku.find_all('tr')

        for i in radau:
            aha = i.find_all('td')
            duomenys[aha[0].text.strip().lower()]= aha[1].text.strip().lower()
                  
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('dronai', 'pigudronai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'dronai', 'pigudronai.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('dronai', 'pigudronai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'dronai', 'pigudronai.db')
        writeMany(duomenys, 'dronai', 'pigudronai.db')
    

driver.close()

In [4]:
# Susirenkame pigu.lt televizoriai nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

nuorodos=[]

for i in range(1, 13):
    url = f'https://pigu.lt/lt/televizoriai?page={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    pigiukai = bs.find_all('div', {'class': 'c-product-card__title'})

    for prekes in pigiukai:
        try:
            nuorodos.append(prekes.find('a')['href'])
        except:
            pass
        
driver.close()

In [5]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    cursor.close()
    connection.close()
    return True


SDB = sqlite3.connect('pigutelevizoriai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists televizoriai
(
nuoroda text,
kaina text
)
 '''
Cs.execute(sql)
Cs.close()
SDB.close()



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)

for nuor in range(0, len(nuorodos)):
    try:
        url = f'{nuorodos[nuor]}'
        driver.get(url)
        time.sleep(2)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')

        stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'pigutelevizoriai.db'))
        duomenys = {key: None for key in stulpeliai}
        duomenys['nuoroda']= nuorodos[nuor]

        try:
            kaina = bs.find('div', {'class': 'c-price h-price--xx-large h-price'})
            kaina = kaina.text.replace(' €','')[:-3].strip()
            duomenys['kaina'] = kaina
        except:
            pass

        try:
            kaina = bs.find('div', {'class': 'c-price h-price--xx-large h-price--new'})
            kaina = kaina.text.replace(' €','')[:-3].strip()
            duomenys['kaina'] = kaina
        except:
            pass
    
        renku = bs.find('div', {'class': 'h-row h-pl--8 h-pr--8'})
        radau= renku.find_all('tr')

        for i in radau:
            aha = i.find_all('td')
            duomenys[aha[0].text.strip().lower()]= aha[1].text.strip().lower()
                  
        
    except:
        stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'pigutelevizoriai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'televizoriai', 'pigutelevizoriai.db')
        pass
    else:
        stulpeliai, *_ =zip(*getColNamesList('televizoriai', 'pigutelevizoriai.db'))
        alterTableAddColumn(duomenys.keys(), stulpeliai, 'televizoriai', 'pigutelevizoriai.db')
        writeMany(duomenys, 'televizoriai', 'pigutelevizoriai.db')
    

driver.close()